In [1]:
# libraries to get data
import urllib.request 
import re 
import io 
import zlib 
import json 
# libraries for nlp datapreprocessing
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
# libraries for nn model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

## Take comments

In [2]:
# take categories
menu_numbers = [] # menu url 
categories = [] # category list
for menu_number in menu_numbers:
    url = "url" + menu_number
    headers = {'headers'}
    request = urllib.request.Request(url, headers=headers)
    try:
        answer = urllib.request.urlopen(request)
        htmlBytes=zlib.decompress(answer.read(), 16+zlib.MAX_WBITS)
        htmlStr = htmlBytes.decode("utf-8")
        json_answer = json.loads(htmlStr)
        
        category_level_1 = [] 
        category_level_2 = [] 
        category_level_3 = [] 
        category_level_4 = []
        
        for item in json_answer["items"]:
            for child in item["children"]:
                if(child["url"]): # url var ise
                    category_level_1.append(child["url"])
                for child2 in child["children"]:
                    if (child2["url"]):
                        category_level_2.append(child2["url"])
                    if (len(child2["children"]) > 0):
                        for child3 in child2["children"]:
                            if (child3["url"]):
                                category_level_3.append(child3["url"])
                                for child4 in child3["children"]:
                                    if (child4["url"]):
                                        category_level_4.append(child4["url"])
        categories.append(category_level_1)
        categories.append(category_level_2)
        categories.append(category_level_3)
        categories.append(category_level_4)

In [4]:
category_list = []  

for category in categories:
    for category1 in category:
        category_list.append(category1.split("?")[0]) # fix ? errors

In [ ]:
with open("category_list.txt", "w") as txt_file: # take backup category urls
    for line in category_list:
        txt_file.write(line + "\n")

In [ ]:
items = set() # set for take items one time
comments = set()

In [ ]:
# take items in categories
for num2 in range(len(category_list)): 
    new_url = url + category_list[num2] 
    
    htmlStr = requests.get(new_url, headers=headers).text

    wanted = '<a href="(.*?)-c-(.*?)sayfa=(.*?)" class="page-(.*?)">(.*?)</a>' 

    pages = re.findall(wanted, htmlStr)
    if len(pages) > 0: 
        page_number = int(pages[-1][2]) 
        page = 1
        for num in range(page_number):
            url2 = new_url + '?sayfa=' + str(num + 1) 
            try:
                htmlStr2 = requests.get(url2, headers=headers).text
            except:
                print("something wrong")

            wanted2 = '"(.*?)"\r\n        data-sku="(.*?)"'  
            pages2 = re.findall(wanted2, htmlStr2)

            for item in pages2:
                if item[0] not in items:
                    items.add(item[0]) 
    else: 
        url3 = new_url
        print(url3)
        try:
            htmlStr3 = requests.get(url3, headers=headers).text
        except:
            print("something wrong")

        wanted3 = '"(.*?)"\r\n        data-sku="(.*?)"'
        pages3 = re.findall(wanted3, htmlStr3)

        for word in pages3:
            if word[0] not in items:
                items.add(word[0])

In [ ]:
with open("item_list_1.txt", "w") as txt_file:  # take backup item urls
    for line in items:
        txt_file.write(line + "\n")

In [ ]:
def turkish_char(word): # fix turkish characters
    return word.replace("&#252;","ü").replace("&#220;","Ü").replace("&#231;","ç").replace("&#199;","Ç").replace("&#246;","ö").replace("&#214;","Ö").replace("&#39;","'")

def nobrackets(word): # fix brackets
    return word.replace("(","").replace(")","")

In [ ]:
for line in items:
    new_url = url + line + "yorumlari"

    htmlStr = requests.get(new_url, headers=headers).text
    page_pattern = '<span class="hermes-PageHolder-module-1QoWq">(.*?)</span>' # page numbers
    pages = re.findall(page_pattern, htmlStr)
    if pages == []:
        page_number = 1
    else:
        page_number = int(pages[-1])

    for page in range(page_number):
        new_url = url + line + "-yorumlari" + "?sayfa=" + str(page+1)

        htmlStr = requests.get(new_url, headers=headers).text


        comm_pattern = '<span itemProp=\\"description\\">(.*?)</span>'  
        comment = re.findall(comm_pattern, htmlStr, re.MULTILINE|re.DOTALL)
        score_pattern ='<span itemProp="ratingValue" content="(.*?)"></span>' 
        score = re.findall(score_pattern, htmlStr, re.MULTILINE|re.DOTALL)

        if (comment != []):
            for i in range(len(comment)):
                comm = comment[i].replace('\n','').replace('\r','').replace('  ','')
                comm = emoji_pattern.sub(r'', comm)
                score1 = score[i]

                comment0 = (score1, nobrackets(turkish_char(comm)))
                comments.add(comment0)

In [ ]:
with open("comment_list_1.csv", "w", encoding="utf-8") as txt_file: # take backup comments
    for line in comments:
        txt_file.write(str(line[0]) + "\t" + str(line[1]).replace("\t", " ") + "\n")

## Data Preprocessing

In [ ]:
data = pd.read_csv("comment_list_1.csv", sep = "\t", names =["Score","Comments"])
df = data.copy()
df.head()

In [3]:
df.isnull().values.sum()

1

In [4]:
df = df.dropna(how='any',axis=0) 

In [5]:
print(df["Score"].value_counts())

5    118181
4     27931
3     12898
1      8484
2      4937
Name: Score, dtype: int64


In [6]:
encoder = LabelEncoder()
Y = encoder.fit_transform(df["Score"])

In [7]:
comments1 = df["Comments"]

In [8]:
 #  remove punctuations
processed = comments1.str.replace(r'[^\w\d\s]', ' ')
 
 #  remove whitespaces
processed = processed.str.replace(r'\s+', ' ')
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [9]:
 #  kower case
processed = processed.str.lower()

In [10]:
 #  remove stopwords

stop_words = set(stopwords.words("turkish"))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [22]:
 #  tokenize

all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [23]:
print(all_words.most_common(15))

[('bir', 76697), ('ürün', 59949), ('güzel', 46639), ('iyi', 37133), ('gayet', 25678), ('ederim', 25090), ('tavsiye', 23612), ('hızlı', 21261), ('kaliteli', 19629), ('geldi', 18598), ('aldım', 17440), ('teşekkürler', 17405), ('kargo', 17381), ('olarak', 14037), ('yok', 13638)]


In [37]:
 #  most common words
word_features = []
for i in range(1500):
    word_features.append(all_words.most_common()[:1500][i][0])

In [ ]:
with open ("common_words.txt", "w", encoding="utf-8") as txt_file: # take most common words
    for line in word_features:
        txt_file.write(line + "\n")

## Build Model

In [4]:
data.head()

,bir,ürün,iyi,güzel,değil,tavsiye,geldi,gayet,aldım,yok,...,lira,piller,gelirsek,yaşıyorum,iyiki,geçmeden,şöyle,sorunlu,mutfak,target
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,0
4,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0


In [6]:
data.values.shape

(17998, 1501)

In [7]:
x = data.drop(["target"], axis=1)

In [8]:
y = data["target"]

In [9]:
x = x.values 
y = y.values

In [10]:
model = tf.keras.models.Sequential()  
model.add(tf.keras.layers.Flatten())  
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))  
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))


In [11]:
# compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
x = np.asarray(x).astype('float32')

In [13]:
# fit model
model.fit(x, y, epochs=3)

Epoch 1/3
563/563 [==============================] - 1s 2ms/step - loss: 0.3837 - accuracy: 0.8333
Epoch 2/3
563/563 [==============================] - 2s 3ms/step - loss: 0.2570 - accuracy: 0.8925
Epoch 3/3
563/563 [==============================] - 1s 2ms/step - loss: 0.1739 - accuracy: 0.9314


In [16]:
word_features_1 = pd.read_csv("common_words.txt", header=None)
word_features = list(word_features_1[0])

In [17]:
 #  find features function

def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

In [18]:
deneme = "çok işime yaradı, beğendim"

In [19]:
deneme1 = find_features(deneme)
aa = []
bb = []
for key, value in deneme1.items() :
    aa.append(key)
    bb.append(value)
temel=pd.DataFrame([bb],columns=aa)
a = model.predict(temel)
if (a[0][0] > a[0][1]+0.1):
    print("olumsuz yorum")
elif(a[0][1] > a[0][0]+0.1):
    print("olumlu yorum")
else:
    print("nötr")

olumlu yorum


In [20]:
print(a) # %86 olumlu 

[[0.13977213 0.86022794]]
